<a href="https://colab.research.google.com/github/ashishjoshi12/Hacakthon2025/blob/main/workflow_gc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#installing ollama
!pip install ollama
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
#starting ollama server locally
import subprocess
import time
process = subprocess.Popen("ollama serve", shell=True)
time.sleep(5)  # Wait for 5 seconds

In [ ]:

#pulling llama3 using ollama
!ollama pull llama3

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏ 1.3 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  28 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  54 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  55 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  57 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  60 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  61 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  64 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a

In [ ]:
#testing llama 3 is availble
!ollama list

In [ ]:
!pip install langchain_community langchain langchain-openai langchain_pinecone langchain[docarray] docarray pydantic==1.10.8 pytube python-dotenv tiktoken pinecone-client scikit-learn ruff pypdf faiss-cpu



Defaulting to user installation because normal site-packages is not writeable
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provid

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ollama 0.4.7 requires pydantic<3.0.0,>=2.9.0, but you have pydantic 1.10.8 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\ajosh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import os
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import DocArrayInMemorySearch
from operator import itemgetter

In [ ]:
import email
from email import policy
from pathlib import Path
import PyPDF2
import os

def read_pdf_content(pdf_path):
    try:
        with open(pdf_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text() + "\n"
            return text
    except Exception as e:
        return f"Error reading PDF: {str(e)}"

def read_msg_file(file_path):
    # Create attachments directory if it doesn't exist
    Path("attachments").mkdir(exist_ok=True)

    # Open the MSG file
    msg = extract_msg.Message(file_path)

    # Print email details
    print("\n=== Email Contents ===")
    print(f"Subject: {msg.subject}")
    print(f"From: {msg.sender}")
    print(f"To: {msg.to}")
    print(f"Date: {msg.date}")
    print("\n=== Email Body ===")
    print(msg.body)

    # Handle attachments
    if msg.attachments:
        print("\n=== Attachments ===")
        for attachment in msg.attachments:
            filename = attachment.getFilename()
            print(f"Found attachment: {filename}")

            # Save attachment to the attachments folder
            attachment_path = Path("attachments") / filename
            with open(attachment_path, "wb") as f:
                f.write(attachment.data)
            print(f"Saved attachment to: {attachment_path}")

            # If attachment is PDF, read and print its content
            if filename.lower().endswith('.pdf'):
                print(f"\n=== PDF Content: {filename} ===")
                pdf_content = read_pdf_content(attachment_path)
                print(pdf_content)
    else:
        print("\nNo attachments found.")

    msg.close()

def read_eml_file(file_path):
    # Create attachments directory if it doesn't exist
    Path("attachments").mkdir(exist_ok=True)

    # Open and parse the EML file
    with open(file_path, 'rb') as f:
        msg = email.message_from_binary_file(f, policy=policy.default)

    # Print email details
    print("\n=== Email Contents ===")
    print(f"Subject: {msg['subject']}")
    print(f"From: {msg['from']}")
    print(f"To: {msg['to']}")
    print(f"Date: {msg['date']}")

    # Handle body and attachments
    for part in msg.walk():
        if part.get_content_maintype() == 'text' and part.get_content_subtype() == 'plain':
            print("\n=== Email Body ===")
            email_body =part.get_content()
            #print(email_body)
            #print(part.get_content())

        elif part.get_content_disposition() == 'attachment':
            filename = part.get_filename()
            if filename:
                print(f"\nFound attachment: {filename}")

                # Save attachment to the attachments folder
                attachment_path = Path("attachments") / filename
                with open(attachment_path, "wb") as f:
                    f.write(part.get_payload(decode=True))
                print(f"Saved attachment to: {attachment_path}")

                # If attachment is PDF, read and print its content
                if filename.lower().endswith('.pdf'):
                    print(f"\n=== PDF Content: {filename} ===")
                    pdf_content = read_pdf_content(attachment_path)
                    #print(pdf_content)
    return email_body+pdf_content

if __name__ == "__main__":
    # Specify the path to your MSG file
    msg_file = "test email.msg"

    try:
        read_msg_file(msg_file)
    except Exception as e:
        print(f"Error: {str(e)}")

    # Specify the path to your EML file
    eml_file = "test email.eml"

    try:
        print(read_eml_file(eml_file))
    except Exception as e:
        print(f"Error: {str(e)}")

Error: name 'extract_msg' is not defined

=== Email Contents ===
Subject: test email
From: ashish joshi <er.ashish.joshi@gmail.com>
To: ashish joshi <er.ashish.joshi@gmail.com>
Date: Sat, 22 Mar 2025 09:31:47 +0000

=== Email Body ===

Found attachment: sample.pdf
Saved attachment to: attachments\sample.pdf

=== PDF Content: sample.pdf ===
This is test email
Sample PDFThis is a simple PDF ﬁle. Fun fun fun.Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Phasellus facilisis odio sed mi. Curabitur suscipit. Nullam vel nisi. Etiam semper ipsum ut lectus. Proin aliquam, erat eget pharetra commodo, eros mi condimentum quam, sed commodo justo quam ut velit. Integer a erat. Cras laoreet ligula cursus enim. Aenean scelerisque velit et tellus. Vestibulum dictum aliquet sem. Nulla facilisi. Vestibulum accumsan ante vitae elit. Nulla erat dolor, blandit in, rutrum quis, semper pulvinar, enim. Nullam varius congue risus. Vivamus sollicitudin, metus ut interdum eleifend, nisi tellus pellen

In [ ]:
# Specify the path to your EML file
eml_file = "test email.eml"
content = read_eml_file(eml_file)
print (content)


=== Email Contents ===
Subject: test email
From: ashish joshi <er.ashish.joshi@gmail.com>
To: ashish joshi <er.ashish.joshi@gmail.com>
Date: Sat, 22 Mar 2025 09:31:47 +0000

=== Email Body ===

Found attachment: sample.pdf
Saved attachment to: attachments\sample.pdf

=== PDF Content: sample.pdf ===
This is test email
Sample PDFThis is a simple PDF ﬁle. Fun fun fun.Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Phasellus facilisis odio sed mi. Curabitur suscipit. Nullam vel nisi. Etiam semper ipsum ut lectus. Proin aliquam, erat eget pharetra commodo, eros mi condimentum quam, sed commodo justo quam ut velit. Integer a erat. Cras laoreet ligula cursus enim. Aenean scelerisque velit et tellus. Vestibulum dictum aliquet sem. Nulla facilisi. Vestibulum accumsan ante vitae elit. Nulla erat dolor, blandit in, rutrum quis, semper pulvinar, enim. Nullam varius congue risus. Vivamus sollicitudin, metus ut interdum eleifend, nisi tellus pellentesque elit, tristique accumsan eros quam

In [ ]:
import re
# Clean unnecessary data in pages

    # Remove headers and footers (example patterns)
content = re.sub(r"Header text pattern", "", content)
content = re.sub(r"Footer text pattern", "", content)

    # Remove specific patterns (example)
content = re.sub(r"© \d{4} Company Name", "", content)

    # Remove page numbers (example)
content = re.sub(r"\d+/\d+", "", content)

    # Remove special characters (except for alphanumeric, spaces, $, ., and ,)
content = re.sub(r"[^a-zA-Z0-9 $,.]", "", content) # Added , and . to the excluded characters

In [ ]:
print (content)

This is test emailSample PDFThis is a simple PDF le. Fun fun fun.Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Phasellus facilisis odio sed mi. Curabitur suscipit. Nullam vel nisi. Etiam semper ipsum ut lectus. Proin aliquam, erat eget pharetra commodo, eros mi condimentum quam, sed commodo justo quam ut velit. Integer a erat. Cras laoreet ligula cursus enim. Aenean scelerisque velit et tellus. Vestibulum dictum aliquet sem. Nulla facilisi. Vestibulum accumsan ante vitae elit. Nulla erat dolor, blandit in, rutrum quis, semper pulvinar, enim. Nullam varius congue risus. Vivamus sollicitudin, metus ut interdum eleifend, nisi tellus pellentesque elit, tristique accumsan eros quam et risus. Suspendisse libero odio, mattis sit amet, aliquet eget, hendrerit vel, nulla. Sed vitae augue. Aliquam erat volutpat. Aliquam feugiat vulputate nisl. Suspendisse quis nulla pretium ante pretium mollis. Proin velit ligula, sagittis at, egestas a, pulvinar quis, nisl.Pellentesque sit amet lect

In [ ]:
pip install langchain_community langchain langchain-openai langchain_pinecone langchain[docarray] docarray pydantic==1.10.8 pytube python-dotenv tiktoken pinecone-client scikit-learn ruff pypdf faiss-cpu

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\ajosh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
pip install pydantic==2.7.4

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ---------------- ----------------------- 0.8/1.9 MB 1.9 MB/s eta 0:00:01
   --------------------------- ------------ 1.3/1.9 MB 2.0 MB/s eta 0:00:01
   --------------------------------- ------ 1.6/1.9 MB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 1.9 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.27.2
    Uninstalling pydantic_core-2.27.2:
      Successfully uninstalled pydantic_core-2.27.2
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.8
    Uninstalling pydantic-1.10.8:
      Successfully uninstalled pydantic-1.10.8
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ollama 0.4.7 requires pydantic<3.0.0,>=2.9.0, but you have pydantic 2.7.4 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\ajosh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
#%pip install langchain

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Adjust based on document size
    chunk_overlap=200,  # Ensures important context isn't lost
    separators=["\n\n", "\n", " "]  # Prioritize splitting at paragraph breaks
)

chunks = text_splitter.split_text(content)

TypeError: ForwardRef._evaluate() missing 1 required keyword-only argument: 'recursive_guard'